<center>
<h1>Get Financial Data From FMP</h1>

## Prep

<center>
<h1>Get Financial Data From FMP</h1>

## Prep

Actually there are a lot of websites providing financial data ([finviz](https://finviz.com/),[yahoo finance](https://finance.yahoo.com/)...). A guide to use finviz to get financial data is given [here](https://towardsdatascience.com/get-up-to-date-financial-ratios-p-e-p-b-and-more-of-stocks-using-python-4b53dd82908f).

Here we obtain data from [FinancialModellingPrep](https://financialmodelingprep.com/) because it provides the most complete data, especially when analyzing historical ratios for the **past few years** for each company. Also, the data can be directly used in [Fundamental Analysis](https://www.investopedia.com/terms/f/fundamentalanalysis.asp) without changing the type.

To be able to get the data you need an API Key from FinancialModellingPrep. Here is the instruction to get the API key:

1. Go to the [Documentation page](https://financialmodelingprep.com/developer/docs/) of FMP
2. Click on "Get my API KEY here"
3. Create a new account or log in

Notice that unpaid account only gives you **limited requests (250)**, you have to upgrade to a premium version or create a new account if you want to keep going. In the code below, I use my own API key to get the result and you should expect an **error** if you don't replace API with you own valid API. 

Note that I am **not** affiliated with FinancialModellingPrep.

## Step By Step Guide to Get Financial Data from FMP

### 1. Import Libraries

In [1]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
import warnings
import json
warnings.filterwarnings('ignore')
# enable pandas to show all the column
pd.set_option('display.max_columns', None)

### 2. [Functions to Get Data](https://financialmodelingprep.com/developer/docs/financial-ratio-free-api/#Python)


In [2]:
def get(url):
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

### 3. Initialize Pandas DataFrame and Store Data from FMP in It

This step is the main step where you can set the list of tickers and the financial data that you need. Here I use tickers of Apple, Google, Amazon and Tesla and get the financial data of FY 2019.

FMP provides all kinds of financial data, which are:
- `Financial Ratios` - Liquidity Measurement Ratios, Profitability Indicator Ratios, Debt Ratios, Operating Performance Ratios, Cash Flow Indicator Ratios and Investment Valuation Ratios
- `Financial Statements` - Income Statement, Balance Sheet Statement and Cash Flow Statement
- `Enterprise Value` - Stock Price, Number of Shares, Market Capitalization...
- ......

Actually this 3 categories almost cover all financial data that we need. For other financial data, you can search by yourself on [Documentation page](https://financialmodelingprep.com/developer/docs/)

Note that there are some financial data that we can not get directly from FMP (e.g. EPS [YoY%](https://www.investopedia.com/terms/y/year-over-year.asp)), we can get the EPS 1 year ago and current EPS to compute the ratio, which I put in "Other_metric".

In [3]:
tickers = ["AAPL", "GOOGL", "AMZN", "TSLA"]
ratio_metric = ["returnOnAssets","priceEarningsRatio","inventoryTurnover"]
IS_metric = ["eps","ebitdaratio","revenue"]
BS_metric = ["totalDebt","totalAssets","totalStockholdersEquity"]
CF_metric = ["capitalExpenditure","operatingCashFlow","dividendsPaid"]
Values_metric = ["marketCapitalization"]
Other_metric = ["eps_1yearago","eps_5yearsago","revenue_1yearago","revenue_5yearsago"]
metric = ratio_metric + IS_metric + BS_metric + CF_metric + Values_metric + Other_metric

df = pd.DataFrame(index=tickers, columns=metric)

In [4]:
def get_fundamental_data(df):
    for symbol in df.index:
        try:
            url_ratios = ("https://financialmodelingprep.com/api/v3/ratios/" + symbol +"?apikey=Your API key")
            url_IS = ("https://financialmodelingprep.com/api/v3/income-statement/" + symbol +"?apikey=Your API key")
            url_BS = ("https://financialmodelingprep.com/api/v3/balance-sheet-statement/" + symbol +"?apikey=Your API key")
            url_CF = ("https://financialmodelingprep.com/api/v3/cash-flow-statement/" + symbol +"?apikey=Your API key")
            url_Values = ("https://financialmodelingprep.com/api/v3/enterprise-values/" + symbol +"?apikey=Your API key")
            
            for i in ratio_metric:                
                df.loc[symbol,i] = get(url_ratios)[0][i]   
            for i in IS_metric:
                df.loc[symbol,i] = get(url_IS)[0][i]
            for i in BS_metric:
                df.loc[symbol,i] = get(url_BS)[0][i]
            for i in CF_metric:
                df.loc[symbol,i] = get(url_CF)[0][i]            
            for i in Values_metric:
                df.loc[symbol,i] = get(url_Values)[0][i]     
                
# It is better to write the financial data below in this order (write 1 year ago first, then 5 years ago) because there 
# are some stocks (e.g. Google) which don't have data of EPS 5 years ago. This order shows financial data as much as
# possible because the loop ends once a NaN appears.
            df.loc[symbol,"eps_1yearago"] = get(url_IS)[1]["eps"]
            df.loc[symbol,"revenue_1yearago"] = get(url_IS)[1]["revenue"]
            df.loc[symbol,"eps_5yearsago"] = get(url_IS)[5]["eps"]
            df.loc[symbol,"revenue_5yearsago"] = get(url_IS)[5]["revenue"]
 
        except Exception as e:
            print (symbol, 'not found')
    return df

In [5]:
df = get_fundamental_data(df)

GOOGL not found


In [6]:
df

,returnOnAssets,priceEarningsRatio,inventoryTurnover,eps,ebitdaratio,revenue,totalDebt,totalAssets,totalStockholdersEquity,capitalExpenditure,operatingCashFlow,dividendsPaid,marketCapitalization,eps_1yearago,eps_5yearsago,revenue_1yearago,revenue_5yearsago
AAPL,0.16323,20.8135,39.4014,11.97,0.293946,260174000000,102067000000,338516000000,90488000000,10495000000,69391000000,-14119000000,1150071557700,12.01,6.49,265595000000,182795000000
GOOGL,0.124472,14.6796,71.968,49.59,0.284276,161857000000,4912000000,275909000000,201442000000,23548000000,54520000000,0,504140484750,44.22,NaN,136819000000,NaN
AMZN,0.0514455,79.7477,8.07611,23.46,0.129509,280522000000,23414000000,225248000000,62060000000,16861000000,38514000000,0,924115920000,20.68,-0.52,232887000000,88988000000
TSLA,-0.0251246,-131.582,5.77393,-4.92,0.0924811,24578000000,2163000000,34309000000,6618000000,1332000000,2405000000,0,1.13423e+11,-5.72,-2.36,21461268000,3198356000
